In [1]:
import json, os, re, requests, subprocess, sys

# if not os.path.exists('python-utils'):
#     subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

# def exec_ipynb(filename_or_url):
#     nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
#     if(nb['nbformat'] >= 4):
#         src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
#     else:
#         src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
#     exec('\n'.join(src), globals())

# exec_ipynb('python-utils/utils.ipynb')
# notebook_wide_display()

In [ ]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')


In [7]:
import bz2, html

STAT_SERVICE_NAME = 'Purpleair download realtime v2'
STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'purpleair-download-realtime v2'

Stat.set_service(STAT_SERVICE_NAME)

In [5]:
import urllib.request, urllib.parse

req = urllib.request.Request(
    "https://map.purpleair.com/token?version=1.8.52", 
    data=None, 
    headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
        'Referer': 'https://map.purpleair.com'
    }
)

response = urllib.request.urlopen(req)
TOKEN = urllib.parse.quote(response.read().decode('utf-8'))

In [8]:
# max_age=300 means look for devices that have uploaded in the last 5 minutes
url = f'https://map.purpleair.com/v1/sensors?token={TOKEN}&fields=name,location_type,latitude,longitude,altitude,last_seen,channel_state,channel_flags,confidence,voc,ozone1,humidity,pressure,temperature,pm2.5,pm2.5_a,pm2.5_b&max_age=300&nwlat=85&selat=-85&nwlng=-180&selng=180'
dest_path = datetime.datetime.utcnow().strftime('mirror2/%Y%m%d/%H%M%Sutc.json.bz2')
response = requests.get(url, timeout=120)
if response.status_code == 200:
    try:
        js = response.json()
    except Exception as ex:
        # print('error parsing as json')
        Stat.warning('Exception %s parsing result from %s' % (ex, url), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        sys.exit()
else:
    # print('Response code %d from %s: %s' % (response.status_code, url, html.escape(response.text)))
    Stat.warning('Response code %d from %s: %s' % (response.status_code, url, html.escape(response.text)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    sys.exit()
# time.sleep(20)

os.makedirs(os.path.dirname(dest_path), exist_ok=True)
tmp_path = '%s-%d.tmp' % (dest_path, os.getpid())
bz2.open(tmp_path, 'wb').write(response.content)
os.rename(tmp_path, dest_path)
Stat.up('Received and compressed %d devices (%.1f KB) to %s (%.1f KB)' % (len(js['data']), len(response.content)/1e3, dest_path, os.path.getsize(dest_path)/1e3),
        host=STAT_HOSTNAME, shortname=STAT_SHORTNAME, valid_for_secs=5*60)
# sleep_until_next_period(150)